In [12]:
from pathlib import Path
dir=Path("/home/davide/Desktop/genomiChro/proteine_RefSeq/ortologhiCCMEE29")
dir_annotati=Path("/home/davide/Desktop/genomiChro/annotati_Refseq")
#extract the name of the files without the extension from dir_annotati
files=[file.stem for file in dir_annotati.iterdir() if file.is_file() and file.suffix==".gbff"]
#iterate over all files in the directory and make a read_csv with "\t" separator and make merge with the previous one on the first column
import pandas as pd
import numpy as np
import os

dfs=[]
for file in dir.iterdir():
    if file.is_file():
        df=pd.read_csv(file, sep="\t", header=None)
        #pick only the first two columns
        df=df.iloc[:,0:2]
        #rename columns
        #get the name of the file without the extension but extracting before _GCF and after sp._
        name=[name for name in files if name in file.stem][0]

        df.columns=["ID", name]
        #find duplicated in ID column and keep only the first one
        df=df[~df.duplicated(subset="ID", keep="first")]
        dfs.append(df)
#merge all the dataframes on ID
df_final=dfs[0]
for df in dfs[1:]:
    df_final=pd.merge(df_final, df, on="ID", how="outer")
#save the final dataframe
df_final


,ID,FACHB-1243,CCNUC1,CCMEE_29,PCC_7203,SAG_39.79,CCALA_051,TS-821,CCALA_043,SAG_2025
0,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_039717719.1,WP_104545246.1,WP_039717719.1,WP_039717719.1
1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_006196724.1,WP_006276978.1,WP_006196724.1,WP_006196724.1
2,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_015153091.1,WP_104546135.1,WP_015153091.1,WP_015153091.1
3,WP_190736661.1,WP_199317029.1,NaN,WP_190736661.1,NaN,WP_106168925.1,WP_106218980.1,NaN,WP_106168925.1,WP_317112279.1
4,WP_250121152.1,NaN,NaN,WP_250121152.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5780,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_015154006.1,WP_104546721.1,WP_015154006.1,WP_015154006.1
5781,WP_275983395.1,NaN,NaN,WP_275983395.1,NaN,NaN,NaN,NaN,NaN,NaN
5782,WP_275983396.1,WP_192156346.1,WP_250017462.1,WP_275983396.1,WP_015163153.1,WP_199755664.1,WP_106545471.1,WP_104545432.1,WP_199755664.1,WP_039719372.1
5783,WP_275983397.1,NaN,NaN,WP_275983397.1,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#now for file in dir_annotati use seqio to extract the coding sequence of all pids in the final dataframe of column with the name of the file
from Bio import SeqIO
diz={}
for file in dir_annotati.iterdir():
    if file.is_file():
        if file.stem=="CCMEE_29":
            continue
        for record in SeqIO.parse(file, "genbank"):
            for feature in record.features:
                if feature.type=="CDS" and feature.qualifiers.get("protein_id",[""])[0] in df_final[file.stem].values:
                    ID=df_final[df_final[file.stem]==feature.qualifiers["protein_id"][0]]["ID"].values[0]
                    #diz[ID]=feature.location.extract(record).seq if ID not in diz else append to list
                    if ID not in diz:
                        diz[ID]=[str(feature.location.extract(record).seq)]
                    else:
                        diz[ID].append(str(feature.location.extract(record).seq))


In [23]:
print(len(diz))

4256


In [26]:
from collections import Counter
counts=Counter([len(value) for value in diz.values()])
#plot the distribution of counts
print(counts)


Counter({8: 2352, 7: 688, 1: 456, 2: 182, 6: 165, 3: 145, 4: 139, 5: 114, 9: 8, 15: 2, 17: 2, 10: 1, 19: 1, 11: 1})


In [27]:
#export the dictionary to a file
import pickle
with open("dizcdsortologhi.pkl", "wb") as f:
    pickle.dump(diz, f)

In [2]:
#read scorescoding.pkl
import pickle
with open("scorescoding.pkl", "rb") as f:
    scores=pickle.load(f)
len(scores)

4256

In [ ]:
len(scores)